# EfficientMed


In [1]:
# hide
import sys
sys.path.append("..")
from nbdev.showdoc import *

Try to optimize network for medical data. Network should have skip connections and self attention (a lot of self attention), so that it can focus on small cancer lesions. 

In [ ]:
# export
class MedConvBlock(nn.Module):
    """Adapted Mobile Inverted Residual Bottleneck Block"""
    
    def __init__(self, 
                 n_inp, # number of input channels
                 n_out, # number of output channels
                 kernel_size, # size of convolution kernel
                 stride, # stride of kernel
                 se_ratio, # squeeze-expand ratio
                 id_skip, # if skip connection shouldbe used
                 expand_ratio, # expansion ratio for inverted bottleneck
                 drop_connect_rate = 0.2, # percentage of dropped connections
                 act_cls=nn.SiLU, # type of activation function
                 norm_type=NormType.Batch, # type of batch normalization
                 **kwargs # further arguments passed to `ConvLayerDynamicPadding`
                ):
        super().__init__()
        store_attr()
        
        # expansion phase (inverted bottleneck)
        n_intermed = n_inp * expand_ratio  # number of output channels
        if expand_ratio != 1: 
            self.expand_conv = ConvLayer(ni=n_inp, nf=n_intermed, 
                                                       ks = 1,norm_type=norm_type, 
                                                       act_cls=act_cls, **kwargs)
            
        # depthwise convolution phase, groups makes it depthwise
        self.depthwise_conv = ConvLayer(ni=n_intermed, nf=n_intermed, 
                                                      groups=n_intermed, ks=kernel_size, 
                                                      stride=stride, norm_type=norm_type, 
                                                      act_cls=act_cls, **kwargs)

        # squeeze and excitation layer, if desired
        self.has_se = (se_ratio is not None) and (0 < se_ratio <= 1)
        if self.has_se:
            num_squeezed_channels = max(1, int(n_inp * se_ratio))
            self.squeeze_expand = nn.Sequential(
                ConvLayer(ni=n_intermed, nf=num_squeezed_channels, ks=1, 
                                        act_cls=act_cls, norm_type=None, **kwargs), 
                ConvLayer(ni=num_squeezed_channels, nf=n_intermed, ks=1,  
                                        act_cls=None, norm_type=None,**kwargs))

        # pointwise convolution phase
        self.project_conv = ConvLayerDynamicPadding(ni=n_intermed, nf=n_out, ks=1, 
                                                    act_cls = None, **kwargs)
        self.drop_conncet = DropConnect(drop_connect_rate)
    
    def forward(self, x):
        if self.id_skip: inputs = x # save input only if skip connection 
        
        # expansion 
        if self.expand_ratio != 1: x = self.expand_conv(x)
        
        # depthwise convolution
        x = self.depthwise_conv(x)

        # squeeze and excitation (self attention)
        if self.has_se:
            x_squeezed = F.adaptive_avg_pool3d(x, 1)
            x_squeezed = self.squeeze_expand(x_squeezed)
            x = x * x_squeezed.sigmoid() # inplace saves a bit of memory
    
        # pointwise convolution
        x = self.project_conv(x)

        # skip connection and drop connect
        if self.id_skip and self.stride == 1 and self.n_inp == self.n_out:
            x = self.drop_conncet(x) + inputs  # skip connection
        return x

In [4]:
from fastai.layers import *
from fastai.basics import *

In [19]:
class SEBlock(Module):
    def __init__(self):
        self.pooling = nn.AdaptiveAvgPool3d(1)
        self.conv1 = nn.Conv3d(80, 10, kernel_size=1)
        self.act1 = nn.ReLU()
        self.conv2 = nn.Conv3d(10, 80, kernel_size=1)
        self.act2 = nn.Sigmoid()
        
    def forward(self, x):
        x_squeezed = self.pooling(x)
        x_squeezed = self.conv1(x_squeezed)
        x_squeezed = self.act1(x_squeezed)
        x_squeezed = self.conv2(x_squeezed)
        x_squeezed = self.act2(x_squeezed)
        x = x * x_squeezed
        return x

In [27]:
SEBlock()(torch.randn(1, 80, 10, 50, 50)).size()

torch.Size([1, 80, 10, 50, 50])

In [ ]:
squeeze_expand = nn.Sequential(
                ConvLayer(ni=n_intermed, nf=num_squeezed_channels, ks=1, 
                                        act_cls=act_cls, norm_type=None, **kwargs), 
                ConvLayer(ni=num_squeezed_channels, nf=n_intermed, ks=1,  
                                        act_cls=None, norm_type=None,**kwargs))

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 01_basics.ipynb.
Converted 02_preprocessing.ipynb.
Converted 03_transforms.ipynb.
Converted 04_dataloaders.ipynb.
Converted 05_learner.ipynb.
Converted 06a_models.alexnet.ipynb.
Converted 06b_models.resnet.ipynb.
Converted 06c_models.densenet.ipynb.
Converted 06d_models.DynamicUnet.ipynb.
Converted 06d_models.unet.ipynb.
Converted 06e_models.deeplabv3.ipynb.
Converted 06f_models.losses.ipynb.
Converted 07_callback.ipynb.
Converted 99_tools.ipynb.
Converted index.ipynb.
